In [2]:
import numpy as np
import pandas as pd

In [7]:
# Read the basic data frame of IMDb
df = pd.read_csv("../dat/imdb_data/title_basics/data.tsv", sep="\t")
print("overall: ", len(df.index))

# Sort out any non-movies (e.g tv-shows)
df = df[df["titleType"] == "movie"]
print("Number of movies:\t", len(df.index))

# Read the review data frame of IMDb
df_reviews = pd.read_csv("../dat/imdb_data/title_ratings/data.tsv", sep="\t")

df["tconst"] = df["tconst"].astype(str)
df_reviews["tconst"] = df_reviews["tconst"].astype(str)

# inner merge of movies and ratings (movies without any votes are dropped)
df = df.merge(df_reviews, how="inner", on="tconst")
print("Number of movies with rating:\t", len(df.index))

# Free up some memory
del df_reviews


# Read in our scraped data
df_scrape = pd.read_csv("../dat/tconst_scraped_data.csv")
df_scrape = df_scrape[~df_scrape.duplicated(['tconst'], keep="first")]
print("Number scraped movies:\t", len(df_scrape))

## Hard Coding

# Change Movie ID "" in the basic DF to the new id ""
# These are the same movie. The basic IMDb data set has an old (invalid) tconst
df["tconst"] = df["tconst"].replace(["tt11905872"], "tt4131756")
df["tconst"] = df["tconst"].replace(["tt4332782"], "tt0246007")
df["tconst"] = df["tconst"].replace(["tt5072702"], "tt4508986")
df["tconst"] = df["tconst"].replace(["tt6419536"], "tt4481310")

df = df[~df.duplicated(['tconst'], keep="first")]

# Drop Movie 
# "tt7368158", "tt2437136", "tt2584608", "tt6858500",
# "tt7375242", "tt7598832", "tt7718552", "tt7728678", "tt7738378"
# "tt8768374", "tt9828428"
# because it's no longer available
# Movie not available (404 Error)
df = df[df.tconst != "tt7368158"]
df = df[df.tconst != "tt2437136"]
df = df[df.tconst != "tt2584608"]
df = df[df.tconst != "tt6858500"]
df = df[df.tconst != "tt7375242"]
df = df[df.tconst != "tt7598832"]
df = df[df.tconst != "tt7718552"]
df = df[df.tconst != "tt7728678"]
df = df[df.tconst != "tt7738378"]
df = df[df.tconst != "tt8768374"]
df = df[df.tconst != "tt9828428"]
print("Number of movies after dropping:\t", df.shape[0])

# Movie "tt1027755" does not have start year but was realeased in 2012
df.iloc[147505, 5] = "2012"
df.iloc[148639, 5] = "2020"
df.iloc[161518, 5] = "2019"
df.iloc[161520, 5] = "2020"
df.iloc[178919, 5] = "2021"
df.iloc[185090, 5] = "2021"
df.iloc[254051, 5] = "2019"
df.iloc[259152, 5] = "2018"
df.iloc[259650, 5] = "2018"
df.iloc[271440, 5] = "2018"
df.iloc[271532, 5] = "2016"
df.iloc[272545, 5] = "2019"

df["tconst"] = df["tconst"].astype(str)
df_scrape["tconst"] = df_scrape["tconst"].astype(str)
df2 = df.copy()


# Merge the data frame and the scraped content
df = df.merge(df_scrape, how="inner", on="tconst")
print("Number of movies after merge:\t", df.shape[0])

# Free up some memory
del df_scrape

# Read the review data frame of IMDb
df_crew = pd.read_csv("../dat/imdb_data/title_crew/data.tsv", sep="\t")

df["tconst"] = df["tconst"].astype(str)
df_crew["tconst"] = df_crew["tconst"].astype(str)

# inner merge of movies and ratings (movies without any votes are dropped)
df = df.merge(df_crew, how="inner", on="tconst")
print("Number of movies after crew merge:\t", len(df))


# Free up some memory
del df_crew

# Sort according to tconst
df = df.sort_values("tconst")

df.to_csv("../dat/raw_data.csv", index=False)
print("saved dataframe sucesfully!")

/home/daniel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


overall:  8598896
Number of movies:	 598851
Number of movies with rating:	 273557


/home/daniel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Number scraped movies:	 273543
Number of movies after dropping:	 273543
Number of movies after merge:	 273543
Number of movies after crew merge:	 273543
saved dataframe sucesfully!


In [ ]:
import re
from collections import Counter

budget = df["Budget"].to_numpy()

currencies = []
counter = 0

for i in range(len(budget)):
    string = budget[i]

    if not pd.isna(string):
        number = "".join(re.findall(r"[\d]+", string))
        currency = re.findall(r"[^{\d,\xa0}]+", string)[0]
        currencies.append(currency)

        if currency in ["$", "£", "€", "CA$", "₹", "A$"]:
            counter += 1

print(Counter(currencies))
print(counter)

# data for ["$", "€", "CA$", "£", "A$", ]

In [ ]:
from currency_converter import CurrencyConverter
from datetime import date  # datetime works too

c = CurrencyConverter(fallback_on_wrong_date=True)
c.convert(100, "USD", date=date(1999, 3, 1))

In [ ]:
np.unique(currencies)

In [ ]:
print(df_budget[df_budget["primaryTitle"] == "Up"])

In [ ]:
print("\xa0")

In [ ]:
# ours 797_615_189.873417
# infl 797_701_090.78

In [ ]:
df.to_csv("../dat/raw_data.csv", index=False)

In [ ]:
c.currencies

In [6]:
df[(df.startYear == "\\N")]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Budget,Gross US & Canada,Opening weekend US & Canada,Gross worldwide,Rating,Critic reviews,User reviews,directors,writers
39315,tt0073399,movie,Atlantic City Jackpot,The Money,0,\N,\N,88,"Action,Drama",5.6,58,NaN,NaN,NaN,NaN,NaN,NaN,4.0,nm0941457,nm0941457
49465,tt0090238,movie,Unternehmen Geigenkasten,Unternehmen Geigenkasten,0,\N,\N,82,"Crime,Family",7.5,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nm0295431,"nm0295431,nm0314329,nm0333146"
52483,tt0095424,movie,Kalamazoo,Kalamazoo,0,\N,\N,88,"Fantasy,Romance",5.6,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nm0285471,"nm0285471,nm0545956"
88852,tt0203086,movie,Home-Coming in Granddaughter: The Cultural Com...,San Jie Cao,0,\N,\N,58,Documentary,7.4,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nm0508629,\N
137508,tt0491947,movie,Ceza,Ceza,0,\N,\N,86,\N,4.8,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nm0351645,nm2137632
137817,tt0494921,movie,Ask Adasi,Ask Adasi,0,\N,\N,\N,Drama,5.7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nm0433657,nm0433657
137837,tt0494975,movie,Nikah,Nikah,0,\N,\N,\N,Drama,4.8,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nm0433657,nm1759633
146760,tt10172414,movie,You Don't Know Nicotine,You Don't Know Nicotine,0,\N,\N,\N,Documentary,8.5,33,NaN,NaN,NaN,NaN,NaN,NaN,3.0,nm7618090,\N
147505,tt1027755,movie,Tied in Blood,Tied in Blood,0,\N,\N,88,"Drama,Horror,Mystery",3.5,222,"£150,000 (estimated)",NaN,NaN,NaN,NaN,9.0,11.0,nm2652936,nm4894856
148639,tt10425364,movie,Samhain,Samhain,0,\N,\N,78,"Comedy,Horror",2.2,6,"$3,500 (estimated)",NaN,NaN,NaN,NaN,NaN,NaN,nm10736500,nm9502936
